In [1]:
!pip install openai requests tiktoken numpy

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
   ---------------------------------------- 0.0/644.8 kB ? eta -:--:--
   ---------------------------------------- 644.8/644.8 kB 6.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.0 kB ? eta -:--:--
   --------------------------------------- 894.0/894.0 kB 13.4 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)


## Create Dataset

In [32]:
import pandas as pd
import dotenv
import json
dotenv.load_dotenv()

True

In [33]:
dataset = pd.read_csv("data/balanced-merged-data.csv")

In [34]:
dataset.head()

,text,label
0,he said this was typical faggot behaviour whic...,good
1,i have no respect for people who write things ...,good
2,you and all the other disabled people are a bl...,good
3,your obvious contempt for immigrants has to st...,good
4,its always him that is the dumb one,good


Fine-tuning gpt-4o-mini-2024-07-18 requires a specially formatted JSONL training file. OpenAI provides the following example in their documentation:

In [35]:
# separate train and validation sets
train, val = dataset.sample(frac=0.8), dataset.sample(frac=0.2)

In [36]:
train.to_json("train.jsonl", orient="records", lines=True)
val.to_json("val.jsonl", orient="records", lines=True)

In [37]:
import re
def anonymize_text(text):
    text = re.sub(r'@\w+', '@user', text)          # usernames
    text = re.sub(r'\b[A-Z][a-z]+\b', '[NAME]', text)  # names (very naive)
    return text

train["text"] = train["text"].apply(anonymize_text)
val["text"] = val["text"].apply(anonymize_text)

In [40]:
def to_chat_format(jsonlpath, type="train"):
    # Load dataset
    df = pd.read_json(jsonlpath, lines=True)

    # Define system prompt
    system_prompt = {
        "role": "system",
        "content": (
            "You are an AI assistant specialized in analyzing text for hateful or abusive language. "
            "Your task is to classify whether the input reflects harmful, bullying, or offensive speech in an online context."
            "You are a classifier AI for hate speech. You do not generate any content yourself."
        )
    }

    # Convert each row into the chat-style format
    def row_to_chat_format(row):
        return {
            "messages": [
                system_prompt,
                {"role": "user", "content": row["text"]},
                {"role": "assistant", "content": row["label"], "weight": 1}
            ]
        }

    # Apply transformation
    chat_data = df.apply(row_to_chat_format, axis=1)
    
    # Save to JSONL
    with open(f"formatted_{type}.jsonl", "w") as f:
        for item in chat_data:
            f.write(json.dumps(item) + "\n")



In [41]:
to_chat_format("data/training_set.jsonl", "train")
to_chat_format("data/validation_set.jsonl", "val")

In [21]:
# # Run preliminary checks
# 
# import json
# 
# # Load the training set
# with open('data/train.jsonl', 'r', encoding='utf-8') as f:
#     training_dataset = [json.loads(line) for line in f]
# 
# # Training dataset stats
# print("Number of examples in training set:", len(training_dataset))
# print("First example in training set:")
# for message in training_dataset:
#     print(message)

# # Load the validation set
# with open('data/val.jsonl', 'r', encoding='utf-8') as f:
#     validation_dataset = [json.loads(line) for line in f]
# 
# # Validation dataset stats
# print("\nNumber of examples in validation set:", len(validation_dataset))
# print("First example in validation set:")
# for message in validation_dataset[0]["text"]:
#     print(message)

Number of examples in training set: 1864
First example in training set:
{'text': 'there is no hatred in my heart for you women ', 'label': 'good'}
{'text': 'you must be seriously sad to write things like trans people deserve to die ', 'label': 'good'}
{'text': 'its in rugby players nature to be uncivilised ', 'label': 'good'}
{'text': 'disabled people refuse to respect us im going to make them suffer for that ', 'label': 'bad'}
{'text': 'that monkey is more intelligent than all of you trans people combined ', 'label': 'bad'}
{'text': 'swarms of gay people are infesting our government ', 'label': 'bad'}
{'text': 'death to all capitalists ', 'label': 'good'}
{'text': 'too many t r a n n i e s in this country ', 'label': 'bad'}
{'text': 'women are a pest ', 'label': 'bad'}
{'text': 'artists are parasites to our society ', 'label': 'good'}
{'text': 'ive been around enough trans people to know theyre all complete assholes ', 'label': 'bad'}
{'text': 'there is nothing worse than a blck perso

In [24]:
# Validate token counts

import json
import tiktoken
import numpy as np
from collections import defaultdict

encoding = tiktoken.get_encoding("o200k_base") # default encoding for gpt-4o models. This requires the latest version of tiktoken to be installed.

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

files = ['data/training_set.jsonl', 'data/validation_set.jsonl']

for file in files:
    print(f"Processing file: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 50)

Processing file: data/training_set.jsonl

#### Distribution of total tokens:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of assistant tokens:
min / max: 0, 0
mean / median: 0.0, 0.0
p5 / p95: 0.0, 0.0
**************************************************
Processing file: data/validation_set.jsonl

#### Distribution of total tokens:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of assistant tokens:
min / max: 0, 0
mean / median: 0.0, 0.0
p5 / p95: 0.0, 0.0
**************************************************


In [30]:
import os

In [33]:
print(os.getenv("AZURE_OPENAI_API_KEY"))
print(os.getenv("AZURE_OPENAI_ENDPOINT"))
print(os.getenv("API_VERSION"))

3a6c6a7b-4e4f-4b0d-8d0f-7b0d4f8d0e4f
https://hubdemor3dai7450370013.openai.azure.com/
2024-12-01-preview


In [26]:
!pip install azure-ai-inference

  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
Using cached isodate-0.7.2-py3-none-any.whl (22 kB)


In [42]:
# Upload fine-tuning files
import dotenv
import os
from openai import AzureOpenAI
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

dotenv.load_dotenv()

endpoint = "https://hubdemor3dai7450370013.openai.azure.com/"
model_name = "gpt-4o"
deployment = "gpt-4o"

subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
print(subscription_key)

api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

# client = ChatCompletionsClient(
#     endpoint="https://hubdemor3dai7450370013.services.ai.azure.com/models",
#     credential=AzureKeyCredential(subscription_key)
# )


NkEqQOV4gkjBQ76TBsHaBj6H1TB8RXYyVDqPIncHEEr5j2FdYzcXJQQJ99BAACfhMk5XJ3w3AAAAACOGf4q9


In [43]:

training_file_name = 'data/formatted_train.jsonl'
validation_file_name = 'data/formatted_val.jsonl'

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

training_response = client.files.create(
    file = open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file = open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-9327fa00b20e42debeb198965e47d1bf
Validation file ID: file-9a447e3c6c2b491f8f93616ea3f0af8a


## Create a customized model


In [44]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0125", # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters. 
    seed = 105  # seed parameter controls reproducibility of the fine-tuning job. If no seed is specified one will be generated automatically.
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))

Job ID: ftjob-e07c0f523f8f4fa1948cffb4c77d2653
Status: ftjob-e07c0f523f8f4fa1948cffb4c77d2653
{
  "id": "ftjob-e07c0f523f8f4fa1948cffb4c77d2653",
  "created_at": 1744713119,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "batch_size": -1,
    "learning_rate_multiplier": 1.0,
    "n_epochs": -1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": 105,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-9327fa00b20e42debeb198965e47d1bf",
  "validation_file": "file-9a447e3c6c2b491f8f93616ea3f0af8a",
  "estimated_finish": 1744714797,
  "integrations": null,
  "metadata": null,
  "method": null
}


## Check fine-tuning job status


In [45]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-e07c0f523f8f4fa1948cffb4c77d2653
Status: pending
{
  "id": "ftjob-e07c0f523f8f4fa1948cffb4c77d2653",
  "created_at": 1744713119,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "batch_size": -1,
    "learning_rate_multiplier": 1.0,
    "n_epochs": -1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": 105,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-9327fa00b20e42debeb198965e47d1bf",
  "validation_file": "file-9a447e3c6c2b491f8f93616ea3f0af8a",
  "estimated_finish": 1744714797,
  "integrations": null,
  "metadata": null,
  "method": null
}


In [11]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10)
print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "ftevent-5cd78064615c441baf471aa1690df259",
      "created_at": 1744710820,
      "level": "info",
      "message": "Preprocessing running for file training file.",
      "object": "fine_tuning.job.event",
      "data": null,
      "type": "message"
    },
    {
      "id": "ftevent-737de1aaa8e64369a5565e0eb92c4a7a",
      "created_at": 1744710818,
      "level": "info",
      "message": "Job enqueued. Waiting for jobs ahead to complete.",
      "object": "fine_tuning.job.event",
      "data": null,
      "type": "message"
    }
  ],
  "has_more": false,
  "object": "list"
}


In [12]:
response = client.fine_tuning.jobs.checkpoints.list(job_id)
print(response.model_dump_json(indent=2))

BadRequestError: Error code: 400 - {'error': {'code': 'unexpectedEntityState', 'message': 'Checkpoints can only be obtained after job is in terminal state'}}

## Analyze your customized model

Azure OpenAI attaches a result file named results.csv to each fine-tune job after it completes. You can use the result file to analyze the training and validation performance of your customized model. The file ID for the result file is listed for each customized model, and you can use the Python SDK to retrieve the file ID and download the result file for analysis.

The following Python example retrieves the file ID of the first result file attached to the fine-tuning job for your customized model, and then uses the Python SDK to download the file to your working directory for analysis.

In [ ]:
# Retrieve the file ID of the first result file from the fine-tuning job
# for the customized model.
response = client.fine_tuning.jobs.retrieve(job_id)
if response.status == 'succeeded':
    result_file_id = response.result_files[0]

retrieve = client.files.retrieve(result_file_id)

# Download the result file.
print(f'Downloading result file: {result_file_id}')

with open(retrieve.filename, "wb") as file:
    result = client.files.content(result_file_id).read()
    file.write(result)